In [47]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms
from torchvision import models,datasets
torch.__version__

'1.3.0'

# 4.2.2 Use Tensorboard for visualization in PyTorch

## Introduction to Tensorboard
Tensorboard is a built-in visualization tool of tensorflow. It makes the understanding, debugging and optimization of tensorflow program easier and more efficient by visualizing the information of the log file output by the tensorflow program.
The visualization of Tensorboard relies on the log files output by the tensorflow program, so tensorboard and tensorflow programs run in different processes.
TensorBoard provides us with an extremely convenient and powerful visualization environment. It can help us understand the entire neural network learning process, data distribution, performance bottlenecks, and so on.

Although tensorboard is a built-in visualization tool of tensorflow, they run in different processes, so there are already great gods on Github who have applied tensorboard to Pytorch [link here]( https://github.com/lanpa/tensorboardX)

## Tensorboard installation
First need to install tensorboard

`pip install tensorboard`



~~ Then install tensorboardx ~~

~~ `pip install tensorboardx` ~~
pytorch 1.1 and later versions have built-in SummaryWriter function, so there is no need to install tensorboardx

After the installation is complete, execute independent commands like visdom
`tensorboard --logdir logs` can be started, the default port is 6006, open `http://localhost:6006/` in the browser to see the web page.

What I want to explain here is that the Microsoft Edge browser css will not load, and it will be displayed normally using chrome

## Page
Unlike visdom, tensorboard artificially distinguishes multiple tags for different types, and each tag page represents a different type.
Below we will give a brief introduction based on different page functions. For more details, please refer to the official website
### SCALAR
Summarize and record scalar data, usually used to visualize the accuracy (val acc), loss value (train/test loss), learning rate, weight and bias of each layer during the training process. The change curve of statistics (mean, std, max/min), etc.
### IMAGES
Visualize the training/test images or feature maps used in the current round of training
### GRAPHS
Visualize the structure of the calculation graph and the information on the calculation graph, usually used to show the structure of the network
### HISTOGRAMS
Visualize the value distribution of the tensor and record the histogram of the variables (the statistical tensor changes with the number of iterations)
### PROJECTOR
Full name Embedding Projector high-dimensional vector for visualization

## Use
Before use, please confirm the execution of `tensorboard --logdir logs` and ensure that the `http://localhost:6006/` page can be opened normally

### Image display
First introduce the relatively simple functions, check the images in our training set and data set, here we use ready-made images as a display. Here is a picture of a cat on wikipedia [here](https://en.wikipedia.org/wiki/Cat#/media/File:Felis_silvestris_catus_lying_on_rice_straw.jpg)

Introduce the tensorboardX package

In [48]:
# The references here should also be modified to torch references
#from tensorboardX import SummaryWriter
from torch.utils.tensorboard import SummaryWriter

In [49]:
cat_img = Image.open('./1280px-Felis_silvestris_catus_lying_on_rice_straw.jpg')
cat_img.size

(1280, 853)

This is a 1280x853 picture, let’s change her into a 224x224 picture first, because vgg16 will be used later

In [50]:
transform_224 = transforms.Compose([
        transforms.Resize(224), # Here to explain that Scale has expired, use Resize
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ])
cat_img_224=transform_224(cat_img)

Show pictures in tebsorboard:

In [51]:
writer = SummaryWriter(log_dir='./logs', comment='cat image') # The logs here should be the same as the parameters of --logdir
writer.add_image("cat",cat_img_224)
writer.close()# Perform close refresh immediately, otherwise it will refresh automatically every 120 seconds

Browser visit `http://localhost:6006/#images` to see the picture of the cat
### Update loss function
To update the loss function and training batches, we use the same simulation display as visdom. Here is the SCALAR page of tensorboard

In [52]:
x = torch.FloatTensor([100])
y = torch.FloatTensor([500])

for epoch in range(30):
    x = x * 1.2
    y = y / 1.1
    loss = np.random.random()
    with SummaryWriter(log_dir='./logs', comment='train') as writer: #You can use python's with syntax directly and automatically call the close method
        writer.add_histogram('his/x', x, epoch)
        writer.add_histogram('his/y', y, epoch)
        writer.add_scalar('data/x', x, epoch)
        writer.add_scalar('data/y', y, epoch)
        writer.add_scalar('data/loss', loss, epoch)
        writer.add_scalars('data/data_group', {'x': x,
                                                'y': y}, epoch)



Browser visit `http://localhost:6006/#scalars` to see the graph
### Use PROJECTOR to visualize high-dimensional vectors
The principle of PROJECTOR is to project high-dimensional vectors to a three-dimensional coordinate system (dimension reduction) through PCA, T-SNE and other methods. Embedding Projector reads data from the checkpoint file saved during the running of the model. By default, principal component analysis (PCA) is used to project high-dimensional data into 3D space. You can also select the T-SNE projection method by setting the settings, here is one Simple display.

We still use the mnist code from Chapter 3

In [53]:
BATCH_SIZE=512
EPOCHS=20
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)

In [54]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        # 1,28x28
        self.conv1=nn.Conv2d(1,10,5) # 10, 24x24
        self.conv2=nn.Conv2d(10,20,3) # 128, 10x10
        self.fc1 = nn.Linear(20*10*10,500)
        self.fc2 = nn.Linear(500,10)
    def forward(self,x):
        in_size = x.size(0)
        out = self.conv1(x) #24
        out = F.relu(out)
        out = F.max_pool2d(out, 2, 2) #12
        out = self.conv2(out) #10
        out = F.relu(out)
        out = out.view(in_size,-1)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        out = F.log_softmax(out,dim=1)
        return out
model = ConvNet()
optimizer = torch.optim.Adam(model.parameters())


In [55]:
def train(model, train_loader, optimizer, epoch):
    n_iter=0
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if(batch_idx+1)%30 == 0: 
            n_iter=n_iter+1
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            #相对于以前的训练方法 主要增加了以下内容
            out = torch.cat((output.data, torch.ones(len(output), 1)), 1) # 因为是投影到3D的空间，所以我们只需要3个维度
            with SummaryWriter(log_dir='./logs', comment='mnist') as writer: 
                #使用add_embedding方法进行可视化展示
                writer.add_embedding(
                    out,
                    metadata=target.data,
                    label_img=data.data,
                    global_step=n_iter)

这里节省时间，只训练一次

In [56]:
train(model, train_loader, optimizer, 0)

Train Epoch: 0 [14848/60000 (25%)]	Loss: 0.352312
Train Epoch: 0 [30208/60000 (50%)]	Loss: 0.202950
Train Epoch: 0 [45568/60000 (75%)]	Loss: 0.156494


打开 `http://localhost:6006/#projector` 即可看到效果。

目前测试投影这部分也是有问题的，根据官网文档的代码进行测试，也显示不出来，正在找原因

### 绘制网络结构
在pytorch中我们可以使用print直接打印出网络的结构，但是这种方法可视化效果不好，这里使用tensorboard的GRAPHS来实现网络结构的可视化。
由于pytorch使用的是动态图计算，所以我们这里要手动进行一次前向的传播.

使用Pytorch已经构建好的模型进行展示

In [57]:
vgg16 = models.vgg16(pretrained=True) # 这里下载预训练好的模型
print(vgg16) # 打印一下这个模型

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

在前向传播前，先要把图片做一些调整

In [58]:
transform_2 = transforms.Compose([
    transforms.Resize(224), 
    transforms.CenterCrop((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])
])

使用上一张猫的图片进行前向传播

In [59]:
vgg16_input=transform_2(cat_img)[np.newaxis]# 因为pytorch的是分批次进行的，所以我们这里建立一个批次为1的数据集
vgg16_input.shape

torch.Size([1, 3, 224, 224])

开始前向传播，打印输出值

In [60]:
out = vgg16(vgg16_input)
_, preds = torch.max(out.data, 1)
label=preds.numpy()[0]
label

287

将结构图在tensorboard进行展示

In [61]:
with SummaryWriter(log_dir='./logs', comment='vgg161') as writer:
    writer.add_graph(vgg16, vgg16_input)

对于Pytorch的1.3版本来说，实测 SummaryWriter在处理结构图的时候是有问题的（或者是需要加什么参数，目前我还没找到），所以建议大家继续使用tensorboardx。

In [ ]:
def train(model, train_loader, optimizer, epoch):
    n_iter=0
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if(batch_idx+1)%30 == 0:
            n_iter=n_iter+1
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            #Compared to the previous training method, the following content is mainly added
            out = torch.cat((output.data, torch.ones(len(output), 1)), 1) # Because it is projected into 3D space, so we only need 3 dimensions
            with SummaryWriter(log_dir='./logs', comment='mnist') as writer:
                #Use add_embedding method for visual display
                writer.add_embedding(
                    out,
                    metadata=target.data,
                    label_img=data.data,
                    global_step=n_iter)

Save time here, only train once

In [ ]:
train(model, train_loader, optimizer, 0)

Open `http://localhost:6006/#projector` to see the effect.

At present, there is a problem with the test projection. According to the code of the official website document, it can’t be displayed. I am looking for the reason.

### Draw network structure
In pytorch, we can use print to directly print out the structure of the network, but this method has a poor visualization effect. Here we use GRAPHS of tensorboard to visualize the network structure.
Since pytorch uses dynamic graph calculation, we have to manually perform a forward propagation here.

Use Pytorch's built model for display

In [ ]:
vgg16 = models.vgg16(pretrained=True) # download the pretrained model here
print(vgg16) # Print this model

Before forwarding, make some adjustments to the picture

In [ ]:
transform_2 = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])
])

Use the previous cat picture for forward propagation

In [ ]:
vgg16_input=transform_2(cat_img)[np.newaxis]# Because pytorch is carried out in batches, we will create a data set with batch 1 here
vgg16_input.shape

Start forward propagation, print out the value

In [ ]:
out = vgg16(vgg16_input)
_, preds = torch.max(out.data, 1)
label=preds.numpy()[0]
label

Display the structure diagram on tensorboard

In [ ]:
with SummaryWriter(log_dir='./logs', comment='vgg161') as writer:
    writer.add_graph(vgg16, vgg16_input)

For the 1.3 version of Pytorch, the measured SummaryWriter has problems when processing the structure diagram (or what parameters need to be added, I haven't found it yet), so I suggest you continue to use tensorboardx.